In [1]:
pip install gradio

  Using cached gradio-4.25.0-py3-none-any.whl (17.1 MB)
  Using cached ffmpy-0.3.2-py3-none-any.whl
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached orjson-3.10.0-cp39-none-win_amd64.whl (139 kB)
  Using cached altair-5.3.0-py3-none-any.whl (857 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached gradio_client-0.15.0-py3-none-any.whl (313 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached fastapi-0.110.1-py3-none-any.whl (91 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl (38 kB)
  Using cached uvicorn-0.29.0-py3-none-any.whl (60 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl (22 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached ruff-0.3.5-py3-none-win_amd64.whl (8.5 MB)
  Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
  Using cached websockets-11.0.3-cp39-cp39-win_amd64.whl (124 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached

In [6]:

from PIL import Image
import tensorflow as tf
import gradio as gr
import io
import base64
from transformers import pipeline
import numpy as np


In [4]:
def evaluate_model(model, test_data):
    results = {}    
    predicted_labels = []
    true_labels = []
    all_images = []
    for images, labels in test_data:
        true_labels.extend(labels.numpy())
        predicted_labels.extend(tf.argmax(model.predict(images), axis=1).numpy())

    # Accuracy
    results['accuracy'] = accuracy_score(true_labels, predicted_labels)
    
    # F1 Score
    results['f1_score'] = f1_score(true_labels, predicted_labels)
        
    cm = confusion_matrix(true_labels, predicted_labels)
    
    class_names = test_data.class_names
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()
    
    return results

In [20]:
def predict_image(img, img_height, img_width, model):
    # Preprocess the image
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.image.resize(img, (img_height, img_width))
    img = np.expand_dims(img, axis=0)
    # img = img / 255.0
    # Predict the class probabilities for the image
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])
    if predicted_class == 0:
        result = "it's a flip"
    elif predicted_class == 1:
        result = "it's not"
    print(result)

In [17]:

def get_data():
    img_height, img_width, img_channels = 180, 180, 3

    batch_size=32
    training="C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project4\\data\\images\\training"
    testing="C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project4\\data\\images\\testing"


    loading_dataste_seed = 0

    train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=loading_dataste_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size

    )


    val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=loading_dataste_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size

    )


    test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    seed=loading_dataste_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size

    )

    classes = train_data.class_names
    shape = (img_height, img_width, img_channels)
    data = (train_data, val_data, test_data)

    return shape, classes, data

In [21]:
def main():
    
    img_height, img_width = 180, 180
    model = tf.keras.models.load_model(r'C:\Users\agnes\Documents\apziva_ai_residency\project4\models\fd_model_v5.h5')

    def predict_image_wrapper(image):
        return predict_image(image, img_height, img_width, model)

    inputs = gr.Image()
    outputs = gr.Textbox()

    # Create a Gradio interface with the predict_image function
    gr.Interface(fn=predict_image_wrapper, 
                 inputs=inputs, 
                 outputs=outputs, 
                 examples=[r'C:\Users\agnes\Documents\apziva_ai_residency\project4\data\images\training\notflip\0001_000000017.jpg',
                           r'C:\Users\agnes\Documents\apziva_ai_residency\project4\data\images\testing\notflip\0002_000000027.jpg', 
                           r'C:\Users\agnes\Documents\apziva_ai_residency\project4\data\images\training\notflip\0004_000000013.jpg'], 
                 title="Image Classification"
                 ).launch()



In [22]:
if __name__ == '__main__':
    main()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 311ms/step
it's a flip
1/1 [==============================] - 0s 110ms/step
it's not
1/1 [==============================] - 0s 100ms/step
it's not
